In [1]:
! pip install pinecone-client pandas datasets --upgrade
! pip install langchain
! pip install langchain-community
! pip install langchain-openai
! pip install spacy
! pip install python-dotenv
! pip install dotenv-python
! pip install python-dotenv --upgrade

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [119]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [120]:
import pandas as pd

df = pd.read_csv("big_basket_beauty.csv")
df.head(2)

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
1,103,Fructis Serum - Long & Strong,Beauty & Hygiene,Hair Care,Garnier,230.0,230.0,Shampoo & Conditioner,4.8,Garnier Fruits Long & Strong Strengthening Ser...


In [121]:
# SOME OF THE VARIABLES WE MIGHT WANT TO EXPERIMENT WITH
CHUNK_SIZE = 400
CHUNK_OVERLAP = 50
EMBEDDING_MODEL = "text-embedding-3-small"

In [126]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
index_name = f"beauty-rc-cs{CHUNK_SIZE}-{EMBEDDING_MODEL}-2"

chunks = text_splitter.create_documents(df.description.values, metadatas=[{"product_name": i} for i in df["product"].values] )
len(chunks), text_splitter._chunk_size, text_splitter._chunk_overlap, index_name

(982, 400, 50, 'beauty-rc-cs400-text-embedding-3-small-2')

In [127]:
def add_product_name_to_page_content(chunk):
    chunk.page_content = f"Product name: {chunk.metadata['product_name']}\n{chunk.page_content}"
    chunk.metadata = {}

for chunk in chunks:
    add_product_name_to_page_content(chunk)
    
print(chunks[0].page_content)

Product name: Creme Soft Soap - For Hands & Body
Nivea Creme Soft Soap gives your skin the best care that it must get. The soft bar consists of Vitamins F and Almonds which are really skin gracious and help you get great skin. It provides the skin with moisture and leaves behind flawless and smooth skin. It makes sure that your body is totally free of germs & dirt and at the same time well nourished.For Beauty tips, tricks & more visit


In [128]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_community.vectorstores import Pinecone as langchain_pinecone

# instantiate a Pinecone client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# First, check if our index already exists and delete stale index
if index_name in [index_info['name'] for index_info in pc.list_indexes()]:
    pc.delete_index(index_name)

# we create a new index
pc.create_index(name=index_name, 
                metric="cosine", 
                dimension=1536, # Dimensions of OpenAI Embedding Model
                spec=ServerlessSpec(
                    cloud="aws",
                    region="us-west-2"
                ) )

docsearch = langchain_pinecone.from_documents(chunks, embeddings, index_name=index_name)

In [129]:
# connect to index
index = pc.Index(index_name)
stats = index.describe_index_stats()
stats

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 982}},
 'total_vector_count': 982}

In [130]:
query = "Is Taft Ultimate Wax a gel or a wax?"
docs = docsearch.similarity_search(query, k=10)
docs

[Document(page_content='Product name: Taft Ultimate Wax\nUltimate strongest taffeta-stop for 100% fixation with liquid crystal gloss effect that gives the hair a luxurious radiance. Ultimate Wax stop that is structured like wax and holds like a gel. It helps to protect the hair from drying out. It has the highest taffeta hold and crystal shine effect.'),
 Document(page_content='Product name: Taft Shine Gel Wax\nTaft Hair Wax is a sculpting product in the Schwarzkopf line and it is used to add hold and shine to hair. The base formula includes vegetable or mineral oils that work to build a finish without being sticky.'),
 Document(page_content="Product name: Taft Ultra Wax\nSchwarzkopf All Weather Taft Ultra Wax builds the perfect finish to your chic hairstyle. Now you don't have to worry about how intact your hairstyle stays in different weather conditions as this hair wax gives the right texture and hold."),
 Document(page_content='Product name: Taft Power Wax\nSchwarzkopf professional